In [0]:
import pandas as pd
import numpy as np

In [86]:
df = pd.read_csv('/content/drive/My Drive/telco-customer-churn.zip')
df['internet'] = df['InternetService']
d = {'No' : 0, 'Yes' : 1, 'Female' : 0, 'Male' : 1, 'DSL': 1, 'Fiber optic': 2, 'No phone service': 2, 'No internet service': 2,
     'Month-to-month': 0, 'One year': 1, 'Two year': 2, 'Electronic check': 0, 'Mailed check': 1, 'Bank transfer (automatic)': 2, 'Credit card (automatic)': 3}
for i in ['gender',	'Partner',	'Dependents',	'PhoneService',	'MultipleLines',	'InternetService',	'OnlineSecurity',	'OnlineBackup', 'DeviceProtection',	'TechSupport',
          'StreamingTV',	'StreamingMovies',	'Contract',	'PaperlessBilling',	'PaymentMethod', 'Churn', 'internet']:
          df[i] = df[i].map(d)
          df[i].astype('float64').dtypes
del df['customerID']
del df['TotalCharges']
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn,internet
0,0,0,1,0,1,0,2,1,0,1,0,0,0,0,0,1,0,29.85,0,1
1,1,0,0,0,34,1,0,1,1,0,1,0,0,0,1,0,1,56.95,0,1
2,1,0,0,0,2,1,0,1,1,1,0,0,0,0,0,1,1,53.85,1,1
3,1,0,0,0,45,0,2,1,1,0,1,1,0,0,1,0,2,42.30,0,1
4,0,0,0,0,2,1,0,2,0,0,0,0,0,0,0,1,0,70.70,1,2


Preparing of data:

In [0]:
from sklearn.utils import shuffle

df = shuffle(df).reset_index(drop=True)

for i in df.columns:
  df[i] = pd.DataFrame(np.asarray(df[i].astype('float64')))
  #df[i] = df[i] / df[i].max()
mean_tenure = np.mean(df['tenure'])
std_tenure = np.std(df['tenure'])
mean_mon_ch = np.mean(df['MonthlyCharges'])
std_mon_ch = np.std(df['MonthlyCharges'])
df['tenure'] = (df['tenure'] - mean_tenure) / std_tenure
df['MonthlyCharges'] = (df['MonthlyCharges'] - mean_tenure) / std_mon_ch

y = df.Churn.copy()
X = df.copy()
del X['Churn']

Design and fit of model:

configuration 1.

In [88]:
X_test = X[:1000]
y_test = y[:1000] 

X_val = X[1000:2000]
y_val = y[1000:2000] 

X_train = X[2000:]
y_train = y[2000:]

from keras import models, layers

model = models.Sequential()
model.add(layers.Dense(8, activation='relu', input_shape=(19,)))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,
                    y_train,
                    epochs=150,
                    batch_size=100,
                    validation_data=(X_val, y_val))
history_dict = history.history

Train on 5043 samples, validate on 1000 samples
Epoch 1/150
5043/5043 [==============================] - 0s 38us/step - loss: 0.5384 - accuracy: 0.7202 - val_loss: 0.4978 - val_accuracy: 0.7430
Epoch 2/150
5043/5043 [==============================] - 0s 15us/step - loss: 0.4905 - accuracy: 0.7303 - val_loss: 0.4785 - val_accuracy: 0.7430
Epoch 3/150
5043/5043 [==============================] - 0s 14us/step - loss: 0.4785 - accuracy: 0.7303 - val_loss: 0.4689 - val_accuracy: 0.7430
Epoch 4/150
5043/5043 [==============================] - 0s 15us/step - loss: 0.4708 - accuracy: 0.7323 - val_loss: 0.4617 - val_accuracy: 0.7450
Epoch 5/150
5043/5043 [==============================] - 0s 14us/step - loss: 0.4648 - accuracy: 0.7496 - val_loss: 0.4563 - val_accuracy: 0.7780
Epoch 6/150
5043/5043 [==============================] - 0s 15us/step - loss: 0.4603 - accuracy: 0.7650 - val_loss: 0.4527 - val_accuracy: 0.7940
Epoch 7/150
5043/5043 [==============================] - 0s 15us/step - loss

In [89]:
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

epochs = list(range(1, len(acc) + 1))

import plotly.graph_objs as go
from plotly.offline import iplot

trace0 = go.Scatter(
    x = epochs,
    y = loss_values,
    mode = 'lines',
    name = 'training loss'
)
trace1 = go.Scatter(
    x = epochs,
    y = val_loss_values,
    mode = 'lines',
    name = 'validation loss'
)
trace2 = go.Scatter(
    x = epochs,
    y = acc,
    mode = 'lines',
    name = 'accuracy'
)
trace3 = go.Scatter(
    x = epochs,
    y = val_acc,
    mode = 'lines',
    name = 'val_accuracy'
)

data_1 = [trace0, trace1]

layout_1 = {'title': 'Config.1 Training and validation loss', 'xaxis': {'title': 'epochs'}, 'yaxis': {'title': 'loss'}}
fig_1 = go.Figure(data=data_1, layout=layout_1)
iplot(fig_1, show_link=False)

data_2 = [trace2, trace3]

layout_2 = {'title': 'Config.1 Training and validation accuracy', 'xaxis': {'title': 'epochs'}, 'yaxis': {'title': 'accuracy'}}
fig_2 = go.Figure(data=data_2, layout=layout_2)
iplot(fig_2, show_link=False)

Configuration 2 with weight regularization.

In [90]:
X_val = X[:1000]
y_val = y[:1000] 

X_test = X[1000:2000]
y_test = y[1000:2000] 

X_train = X[2000:]
y_train = y[2000:]

from keras import models, layers, regularizers
import plotly.graph_objs as go
from plotly.offline import iplot

model = models.Sequential()
model.add(layers.Dense(3, kernel_regularizer=regularizers.l2(0.01),
                       activation='relu', input_shape=(19,)))
model.add(layers.Dense(3, kernel_regularizer=regularizers.l2(0.01),
                       activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,
                    y_train,
                    epochs=150,
                    batch_size=100,
                    validation_data=(X_val, y_val), verbose=0)

# Precision and recall on validation:

y_predicted = np.round(model.predict(X_val))
true_positive = 0
false_positive = 0
false_negative = 0
for i in range(len(y_predicted)):
  if y_predicted[i] == 1:
    if y_val[i] == 1:
      true_positive += 1
    else:
      false_positive += 1
  else:
    if y_val[i] == 1:
      false_negative += 1

precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
print('precision= ', precision)
print('recall= ', recall)

# Visualization:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
epochs = list(range(1, len(acc) + 1))

trace0 = go.Scatter(
    x = epochs,
    y = loss_values,
    mode = 'lines',
    name = 'training loss'
)
trace1 = go.Scatter(
    x = epochs,
    y = val_loss_values,
    mode = 'lines',
    name = 'validation loss'
)
trace2 = go.Scatter(
    x = epochs,
    y = acc,
    mode = 'lines',
    name = 'accuracy'
)
trace3 = go.Scatter(
    x = epochs,
    y = val_acc,
    mode = 'lines',
    name = 'val_accuracy'
)

data_1 = [trace0, trace1]

layout_1 = {'title': 'Config.2 with L2-regularization. Training and validation loss', 'xaxis': {'title': 'epochs'}, 'yaxis': {'title': 'loss'}}
fig_1 = go.Figure(data=data_1, layout=layout_1)
iplot(fig_1, show_link=False)

data_2 = [trace2, trace3]

layout_2 = {'title': 'Config.2 with L2-regularization. Training and validation accuracy', 'xaxis': {'title': 'epochs'}, 'yaxis': {'title': 'accuracy'}}
fig_2 = go.Figure(data=data_2, layout=layout_2)
iplot(fig_2, show_link=False)

precision=  0.6666666666666666
recall=  0.5396825396825397


Configuration 3. Design model with K-fold cross-validation.

In [93]:
X_test = X[:1500]
y_test = y[:1500]
X_train = X[1500:]
y_train = y[1500:]

from keras import models, layers, regularizers
import plotly.graph_objs as go
from plotly.offline import iplot

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(4, kernel_regularizer=regularizers.l2(0.01),
                         activation='relu', input_shape=(19,)))
  model.add(layers.Dense(4, kernel_regularizer=regularizers.l2(0.01),
                         activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(optimizer='rmsprop',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model


k = 4
fold_size = len(X_train) // k
num_epochs = 150
all_scores = []

for i in range(k):
  print('processing fold #', i)
  X_val = X_train[i * fold_size: (i + 1) * fold_size]
  y_val = y_train[i * fold_size: (i + 1) * fold_size]

  partial_X_train = np.concatenate(
      [X_train[:i * fold_size],
       X_train[(i + 1) * fold_size:]],
       axis=0)
  partial_y_train = np.concatenate(
      [y_train[:i * fold_size],
       y_train[(i + 1) * fold_size:]],
       axis=0)
  model = build_model()
  history = model.fit(partial_X_train, partial_y_train, epochs=num_epochs,
            batch_size=50, validation_data=(X_val, y_val), verbose=0)
  
  history_dict = history.history
  val_loss_values = history_dict['val_loss']
  val_accuracy_values = history_dict['val_accuracy']
  loss_values = history_dict['loss']
  accuracy_values = history_dict['accuracy']
  epochs = list(range(1, len(accuracy_values) + 1))

  trace0 = go.Scatter(
      x = epochs,
      y = loss_values,
      mode = 'lines',
      name = 'training loss'
  )
  trace1 = go.Scatter(
      x = epochs,
      y = val_loss_values,
      mode = 'lines',
      name = 'validation loss'
  )
  trace2 = go.Scatter(
      x = epochs,
      y = accuracy_values,
      mode = 'lines',
      name = 'accuracy'
  )
  trace3 = go.Scatter(
      x = epochs,
      y = val_accuracy_values,
      mode = 'lines',
      name = 'val_accuracy'
  )

  data = [trace0, trace1, trace2, trace3]

  layout = {'title': 'Config.3 with L2-regularization and K-fold cross-validatioon. Training and validation loss and accuracy. 4 neurons, 150 epochs', 'xaxis': {'title': 'epochs'}, 'yaxis': {'title': 'loss and accuracy'}}
  fig = go.Figure(data=data, layout=layout)
  iplot(fig, show_link=False)

  val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
  all_scores.append(val_acc)


processing fold # 0


processing fold # 1


processing fold # 2


processing fold # 3


In [94]:
np.mean(all_scores)

0.7945848405361176

Final model.

In [95]:
X_test = X[:1500]
y_test = y[:1500]
X_train = X[1500:]
y_train = y[1500:]

from keras import models, layers, regularizers

model = models.Sequential()
model.add(layers.Dense(4, kernel_regularizer=regularizers.l2(0.01),
                       activation='relu', input_shape=(19,)))
model.add(layers.Dense(4, kernel_regularizer=regularizers.l2(0.01),
                       activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=150, batch_size=20, verbose=0)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('test loss= ', test_loss)
print('test accuracy= ', test_acc)

test loss=  0.4006685563723246
test accuracy=  0.8166666626930237


In [96]:
# Precision and recall:

y_predicted = np.round(model.predict(X_test))
true_positive = 0
false_positive = 0
false_negative = 0
for i in range(len(y_predicted)):
  if y_predicted[i] == 1:
    if y_test[i] == 1:
      true_positive += 1
    else:
      false_positive += 1
  else:
    if y_test[i] == 1:
      false_negative += 1

precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
print('precision= ', precision)
print('recall= ', recall)

precision=  0.6690140845070423
recall=  0.5121293800539084
